# Скачиваю офисы с ЦИАН
## 1. Собираю ссылки на страницы с офисами по субъектам (selenium)

In [2]:
# Подгружаем пакеты и настраиваем веб-краулер
from selenium import webdriver
import time

from tqdm import tqdm_notebook
import pickle
import numpy as np
import pandas as pd 
import re

In [ ]:
start_url = 'https://www.cian.ru/cat.php?deal_type=rent&engine_version=2&offer_type=offices&office_type%5B0%5D=1&region=-1'
driver = webdriver.Chrome( )

driver.get(start_url)
time.sleep(2)
buttom = driver.find_element_by_xpath('//*[@id="frontend-serp"]/div/div[1]/div/div/div[2]/div[1]')
buttom.click()
time.sleep(2)

bt2 = driver.find_element_by_xpath('//*[@id="frontend-serp"]/div/div[9]/div/div[2]/div[2]/div/div/div[2]/div/div[1]')
bt2.click()

knopki = driver.find_elements_by_class_name("column-item--3sFX4")
region_names = [item.text for item in knopki]

driver.close() # закрываем браузер 

In [3]:
%%time
url_dict = { }
url_dict_2 = { }

driver = webdriver.Chrome( )
driver.set_window_size(1000, 8000)
# Надо установить величину окна, иначе открывающееся меню
# с регионами не будет влезать и код будет выдавать ошибку
for i in range(len(region_names)):
    driver.get(start_url)
    time.sleep(1)
    buttom = driver.find_element_by_xpath('//*[@id="frontend-serp"]/div/div[1]/div/div/div[2]/div[1]')
    buttom.click()
    time.sleep(2)
    bt2 = driver.find_element_by_xpath('//*[@id="frontend-serp"]/div/div[9]/div/div[2]/div[2]/div/div/div[2]/div/div[1]')
    bt2.click()
    time.sleep(2)
    knopki = driver.find_elements_by_class_name("column-item--3sFX4")
    knopki[i].click()
    time.sleep(3)
    bt3 = driver.find_element_by_xpath('//*[@id="frontend-serp"]/div/div[9]/div/div[2]/div[4]/div/button')
    bt3.click()
    time.sleep(3)
    print(region_names[i],':',driver.current_url)
    url_dict[region_names[i]] = driver.current_url
    # Резервная ссылка 
    try:
        sec_url = driver.find_element_by_class_name("list-itemLink--39icE").get_attribute("href")
        url_dict_2[region_names[i]] = sec_url
        print(sec_url,'\n')
    except:
        print('Только одна страница в ',region_names[i],'\n')
    
driver.close() # закрываем браузер 

Адыгея : https://www.cian.ru/snyat-ofis-adygeya/
Только одна страница в  Адыгея 

Алтай (Республика) : https://www.cian.ru/cat.php?deal_type=rent&engine_version=2&offer_type=offices&office_type%5B0%5D=1&region=4554
Только одна страница в  Алтай (Республика) 

Алтайский край : https://barnaul.cian.ru/snyat-ofis-altayskiy-kray/
https://barnaul.cian.ru/cat.php?deal_type=rent&engine_version=2&offer_type=offices&office_type%5B0%5D=1&p=2&region=4555 

Амурская область : https://www.cian.ru/cat.php?deal_type=rent&engine_version=2&offer_type=offices&office_type%5B0%5D=1&region=4556
Только одна страница в  Амурская область 

Архангельская область : https://www.cian.ru/snyat-ofis-arhangelskaya-oblast/
https://www.cian.ru/cat.php?deal_type=rent&engine_version=2&offer_type=offices&office_type%5B0%5D=1&p=2&region=-1 

Астраханская область : https://astrahan.cian.ru/snyat-ofis-astrahanskaya-oblast/
https://astrahan.cian.ru/cat.php?deal_type=rent&engine_version=2&offer_type=offices&office_type%5B0%5D

In [4]:
# Добавляем Москву и Питер и заменяем пару битых сылок
# При скачке есть ряд проблем с регионами в url_dict_2. Почему-то часть страниц нормально не прогрузились и заменились 
# На Москву. Вручную поменяем все такие ссылки. Ибо 5 ссылок не 85. Ха! 

url_dict_2.update({'Москва' : 'https://www.cian.ru/cat.php?deal_type=rent&engine_version=2&offer_type=offices&office_type%5B0%5D=1&p=2&region=1',
 'Санкт-Петербург' : 'https://spb.cian.ru/cat.php?deal_type=rent&engine_version=2&offer_type=offices&office_type%5B0%5D=1&p=2&region=2'})

url_dict['Мордовия'] = 'https://www.cian.ru/cat.php?deal_type=rent&engine_version=2&offer_type=offices&office_type%5B0%5D=1&region=4592'
url_dict['Ненецкий АО'] = 'https://www.cian.ru/cat.php?deal_type=rent&engine_version=2&offer_type=offices&office_type%5B0%5D=1&region=4595'
url_dict['Ростовская область'] = 'https://rostov.cian.ru/snyat-ofis-rostovskaya-oblast/'
url_dict['Севастополь'] = 'https://www.cian.ru/snyat-ofis-sevastopol/'

print(len(url_dict))

url_dict_2['Архангельская область'] = 'https://www.cian.ru/cat.php?deal_type=rent&engine_version=2&offer_type=offices&office_type%5B0%5D=1&p=2&region=4557'
url_dict_2['Кабардино-Балкария'] = 'https://www.cian.ru/cat.php?deal_type=rent&engine_version=2&offer_type=offices&office_type%5B0%5D=1&p=2&region=4573'
del url_dict_2['Мордовия'] #удалить 
del url_dict_2['Ненецкий АО']  #удалить 
url_dict_2['Ростовская область'] = 'https://rostov.cian.ru/cat.php?deal_type=rent&engine_version=2&offer_type=offices&office_type%5B0%5D=1&p=2&region=4606'
url_dict_2['Севастополь'] = 'https://www.cian.ru/cat.php?deal_type=rent&engine_version=2&offer_type=offices&office_type%5B0%5D=1&p=2&region=184723'

print(len(url_dict_2))

83
52


In [7]:
with open('url_dict_v2', 'wb') as f:
    pickle.dump(url_dict, f)
    
with open('url_dict_2_v2', 'wb') as f:
    pickle.dump(url_dict_2, f)

## 2. Скачиваем по каждому городу все возможные ссылки на офисы 

In [2]:
import requests
import numpy as np 
import pandas as pd
from tqdm import tqdm_notebook # Красивые циклы
from bs4 import BeautifulSoup
from user_agent import generate_user_agent 

In [3]:
# Анонимизация 
# В консоли: pip install PySocks
# Перед запуском кода нужно открыть TOP прямо в браузере 
import socks
import socket
socks.set_default_proxy(socks.SOCKS5, "localhost", 9150)
socket.socket = socks.socksocket
# ТОР по умолчанию идет через сокет 9150

from urllib.request import urlopen
print(urlopen('http://icanhazip.com').read())

# Для того, чтобы изменить частоту смены ip находим в папке, где стоит TOR
# файл torcc и в него добавляем строки: 

# HiddenServiceStatistics 0
# CircuitBuildTimeout 10
# LearnCircuitBuildTimeout 0
# MaxCircuitDirtiness 10

b'192.160.102.170\n'


In [4]:
# Функция отвечающая на вопрос появилась ли на странице капча
def captcha(soup):
    a = False
    try:
        wt = soup.title.text
    except:
        wt = 'ok'
    if wt == 'Captcha - база объявлений ЦИАН':
        a = True
    return a

In [ ]:
# Эксперименты с достопом к странице
page = 'https://www.cian.ru/cat.php?deal_type=rent&engine_version=2&offer_type=offices&office_type%5B0%5D=1&region=4575'
for i in range(10):
    time.sleep(9)
    print(urlopen('http://icanhazip.com').read())
    response = requests.get(page,headers = {'User-Agent':generate_user_agent()})
    html = response.content
    soup = BeautifulSoup(html,'html.parser')
    if captcha(soup):
        print('На странице капча')
        
    tx = soup.find('div',{'class': "nothingFound-title--1c11b"})
    print(tx)

Для некоторых ip почему-то по дефолту вылезает капча. Видимо, нужно прописать отдельный кусок, который будет проверять страницу на её наличие. 

Интересно было бы заодно узнать сколько уникальных ip умеет генерировать TOR 

In [54]:
%%time 
# Качаем субъекты, для которых есть только одна страница с офисами
mp_city = set(url_dict_2.keys())
ip_list = [ ]

offices_hrefs = { }
# Скопировали словарь с городами и будем удалять из него неоткапчённые ссылки
capcha_hrefs = url_dict.copy() 

total1 = 0  # считаем общее число офисов
while len(capcha_hrefs) != 0:
    print('На данной итерации осталось', len(capcha_hrefs))
    bad_city = [ ]
    for city in capcha_hrefs.keys():
        # Если для офиса есть много ссылок, в топку его
        if city in mp_city:
            bad_city.append(city)
            continue
        
        # Закидываем текущий ip в наш список (эксперимент)
        new_ip = urlopen('http://icanhazip.com').read()
        ip_list.append(new_ip)
        ip_list = list(set(ip_list))
        
        page = capcha_hrefs[city]        
        response = requests.get(page)
        html = response.content
        soup = BeautifulSoup(html,'html.parser')
        if captcha(soup):
            print('На странице капча')
            time.sleep(10)
            continue 
        
        dirty_hrefs = soup.find_all('a',{'class':"header-link--22fgv"})
        clean_hrefs = [item.attrs['href'] for item in dirty_hrefs]
        print('В', url_dict[city], len(clean_hrefs), ' а вчистую', len(set(clean_hrefs)))
        offices_hrefs[city] = clean_hrefs
        total1 += len(clean_hrefs)
        bad_city.append(city)
    # Удаляем то, что скачали
    for cty in bad_city:
        del capcha_hrefs[cty]
print(total1)

На данной итерации осталось 83
В https://www.cian.ru/cat.php?deal_type=rent&engine_version=2&offer_type=offices&office_type%5B0%5D=1&region=4631 1  а вчистую 1
В https://www.cian.ru/cat.php?deal_type=rent&engine_version=2&offer_type=offices&office_type%5B0%5D=1&region=187450 2  а вчистую 2
В https://vladivostok.cian.ru/cat.php?deal_type=rent&engine_version=2&offer_type=offices&office_type%5B0%5D=1&region=4604 1  а вчистую 1
В https://www.cian.ru/cat.php?deal_type=rent&engine_version=2&offer_type=offices&office_type%5B0%5D=1&region=4595 25  а вчистую 25
В https://www.cian.ru/cat.php?deal_type=rent&engine_version=2&offer_type=offices&office_type%5B0%5D=1&region=4571 25  а вчистую 25
В https://yanao.cian.ru/cat.php?deal_type=rent&engine_version=2&offer_type=offices&office_type%5B0%5D=1&region=4635 25  а вчистую 25
В https://www.cian.ru/cat.php?deal_type=rent&engine_version=2&offer_type=offices&office_type%5B0%5D=1&region=4628 25  а вчистую 25
На странице капча
В https://www.cian.ru/cat.ph

Неожиданно выяснилось, что иногда возникает вот такая ошибка (я в шоке): 

<html>
<head><title>403 Forbidden</title></head>
<body bgcolor="white">
<center><h1>403 Forbidden</h1></center>
<hr><center>nginx</center>
</hr></body>
</html>

In [69]:
%%time
total2 = 0 

capcha_hrefs = url_dict_2.copy() 
while len(capcha_hrefs) != 0:
    print('\n','На данной итерации осталось', len(capcha_hrefs))
    bad_city = [ ]
    for cty in capcha_hrefs.keys():        
        # Закидываем текущий ip в наш список (эксперимент)
        new_ip = urlopen('http://icanhazip.com').read()
        ip_list.append(new_ip)
        ip_list = list(set(ip_list))
        
        vec = capcha_hrefs[cty].split('p=2')
        page_part_1 = vec[0] + 'p='
        page_part_2 = vec[1]
        page = page_part_1 + str(1) + page_part_2
        
        # Первый запрос поможет узнать сколько всего страниц для этого города существует 
        response = requests.get(page)
        html = response.content
        soup = BeautifulSoup(html,'html.parser')
        if captcha(soup):
            print('На странице капча')
            time.sleep(10)
            continue 
        
        try: # страхуемся от любой криповой фигни вроде ошибок 403
            # Количество объявлений
            nnn = int(re.sub('\D','',soup.find('div',{'class':"totalOffers--yZcBn"}).text))
            print('Для',cty,'есть целых',nnn,'объявлений.')
            n = nnn//25 + (nnn%25 > 0)*1
            # Число просматриваемых объявлений ограничивается 60 =(
            # Для обхода этого ограничения нужен более умный парсер
            if n > 60:
                n = 60

            cur_hrefs = [ ]
            # Внутренний капча-обходчик... 
            # Создаём список из ссылок, которые надо посетить: 
            need_hrefs = [page_part_1 + str(p) + page_part_2  for p in range(1,n+1)]
            while len(need_hrefs) > 0: 
                seen_hrefs = [ ]
                for page in need_hrefs:
                    response = requests.get(page)
                    html = response.content
                    soup = BeautifulSoup(html,'html.parser')                
                    if captcha(soup):
                        time.sleep(10)
                        continue 

                    dirty_hrefs = soup.find_all('a',{'class':"header-link--22fgv"})
                    clean_hrefs = [item.attrs['href'] for item in dirty_hrefs]
                    cur_hrefs.extend(clean_hrefs)
                    seen_hrefs.append(page)

                for seen in seen_hrefs:
                    need_hrefs.remove(seen)
            print('В', capcha_hrefs[cty], len(cur_hrefs), ' а вчистую', len(set(cur_hrefs))) 
            offices_hrefs[cty] = cur_hrefs
            total2 += len(cur_hrefs)
            bad_city.append(cty)
        except:
            print('Нечто странное произошло для сраницы ', page)            
            
    # Удаляем то, что скачали
    for cty in bad_city:
        del capcha_hrefs[cty]
            

На данной итерации осталось 52
Для Иркутская область есть целых 114 объявлений.
В https://irkutsk.cian.ru/cat.php?deal_type=rent&engine_version=2&offer_type=offices&office_type%5B0%5D=1&p=2&region=4572 114  а вчистую 114
На странице капча
Для Кабардино-Балкария есть целых 39 объявлений.
В https://www.cian.ru/cat.php?deal_type=rent&engine_version=2&offer_type=offices&office_type%5B0%5D=1&p=2&region=4573 39  а вчистую 39
На странице капча
Для Тверская область есть целых 189 объявлений.
В https://tver.cian.ru/cat.php?deal_type=rent&engine_version=2&offer_type=offices&office_type%5B0%5D=1&p=2&region=4619 191  а вчистую 191
На странице капча
На странице капча
Для Ивановская область есть целых 88 объявлений.
В https://ivanovo.cian.ru/cat.php?deal_type=rent&engine_version=2&offer_type=offices&office_type%5B0%5D=1&p=2&region=4570 91  а вчистую 89
На странице капча
На странице капча
На странице капча
На странице капча
Для Саратовская область есть целых 79 объявлений.
В https://saratov.cian.ru/c

In [72]:
ttotal = 0 
for key in offices_hrefs:
    #print(len(offices_hrefs[key]))
    ttotal += len(offices_hrefs[key])
print(ttotal)
len(offices_hrefs)

16762


85

In [73]:
# Сохраним словари из ссылок. 
with open('cian_hrefs', 'wb') as f:
    pickle.dump(offices_hrefs, f)

## 3. Пишем функцию для скачки информации по отдельному офису 

In [5]:
with open('cian_hrefs', 'rb') as f:
    offices_hrefs = pickle.load(f)

In [6]:
hrf = [ ]
for key in offices_hrefs:
    hrf.extend(offices_hrefs[key])

# по ходу скачал рекламные ссылки случайно...
print(len(hrf))
print(len(set(hrf)))

hrf = list(set(hrf))

16762
14699


In [37]:
def one_office_download(soup):
    data = { }  # Задали пустой словарь, в который мы будем сохранять данные
    
    # Вытаскиваем цену и адрес
    price1 = soup.findAll('div', attrs = {'class':"object_descr_price"})[0].text.strip()
    data['Цена1'] = price1
    price2 = soup.findAll('div', attrs = {'class':"cf-object-descr-add"})[0].text.strip()
    data['Цена2'] = price2 
    addr = soup.findAll('h1', attrs = {'class':"object_descr_addr"})[0].text.strip()
    data['Адрес'] = addr
    
    # Вытаскиваем координаты квартиры
    coordinates = soup.findAll('div',{'class':"object-descr__map-tabs__content js-object_descr__panorama"})[0]
    data['Координаты'] = coordinates.panorama.attrs['point']
    
    # Вытаскиваем всё остальное
    table = soup.findAll('article',attrs = {'class':"cf-comm-offer-detail"})[0]
    cur_dct = dict(zip([item.text for item in table.find_all('dt')],
                           [jtem.text for jtem in table.find_all('dd')]))
    data.update(cur_dct)
    return(data) 

In [42]:
# Подгружаем страничку с информацией по квартире
response = requests.get('https://spb.cian.ru/rent/commercial/162531122/')
html = response.content
soup = BeautifulSoup(html,'html.parser')  

one_office_download(soup)

{'Адрес': 'Ленинградская область,\n    \n        \n        Кировский район,\n    \n        \n        Кировск,\n    \n        \n        ул. Кирова',
 'Координаты': '[30.991374,59.873166]',
 'Площадь:': '30\xa0м²',
 'Состояние:': 'офисная отделка',
 'Срок аренды:': 'длительный',
 'Тип аренды:': 'прямая',
 'Цена1': '20\xa0000\xa0руб. в\xa0месяц',
 'Цена2': '8\xa0000\xa0руб. за\xa0м² в\xa0годкомиссия\xa0100%',
 'Этаж:': '2\xa0из\xa03'}

In [43]:
captcha(soup)

False

In [ ]:
# Кусок кода на случай, если всё упало 
ip_list = [ ]

with open('cian_data12', 'rb') as f:
    data = pickle.load(f)

print(len(data))
cur_hrefs = hrf.copy()
    
bad_hrefs = [item['href'] for item in data]
    
for hr in bad_hrefs:
    try:
        cur_hrefs.remove(hr)
    except:
        pass
    
print(len(cur_hrefs))

In [51]:
%%time
# Скопировали вектор с хрефами и будем удалять из него неоткапчённые ссылки 
# Закоменечно, так как всё падало и приходилось юзать код выше
#cur_hrefs = hrf.copy()
#cur_hrefs = cur_hrefs
#data = [ ]
save_i = 11
while len(cur_hrefs) != 0:
    print('\n', 'На данной итерации осталось', len(cur_hrefs))
    bad_hrefs = [ ] # Сюда кладём просмотренные 
    for hr in tqdm_notebook(cur_hrefs): 
        # Закидываем текущий ip в наш список (эксперимент)
        new_ip = urlopen('http://icanhazip.com').read()
        ip_list.append(new_ip)
        ip_list = list(set(ip_list))     
        
        response = requests.get(hr)
        html = response.content
        soup = BeautifulSoup(html,'html.parser')  
        if captcha(soup):
            print('На странице капча')
            time.sleep(10)
            continue 
        try:        
            ddt = one_office_download(soup)
            ddt.update({'href' : hr})
            data.append(ddt)
            bad_hrefs.append(hr)
        except:
            print('На странице ошибка')
            continue 
    # На всякий случай сохраним накаченое за итерацию 
    with open('cian_data' + str(save_i), 'wb') as f:
        pickle.dump(data, f)
    save_i += 1    
    # Удаляем то, что уже скачали
    for hr in bad_hrefs:
        cur_hrefs.remove(hr) 


 На данной итерации осталось 411
На странице капча



Exception in thread Thread-7:
Traceback (most recent call last):
  File "/usr/lib/python3.5/threading.py", line 914, in _bootstrap_inner
    self.run()
  File "/usr/local/lib/python3.5/dist-packages/tqdm/_tqdm.py", line 102, in run
    for instance in self.tqdm_cls._instances:
  File "/usr/lib/python3.5/_weakrefset.py", line 60, in __iter__
    for itemref in self.data:
RuntimeError: Set changed size during iteration



На странице капча
На странице капча
На странице капча
На странице капча
На странице капча
На странице капча
На странице капча
На странице капча
На странице капча
На странице капча
На странице капча
На странице капча
На странице капча
На странице капча
На странице капча
На странице капча
На странице капча
На странице ошибка
На странице капча
На странице капча
На странице капча
На странице ошибка
На странице ошибка
На странице ошибка
На странице ошибка
На странице ошибка
На странице капча
На странице капча
На странице капча
На странице капча
На странице капча
На странице капча
На странице капча
На странице капча
На странице капча
На странице капча
На странице капча
На странице капча
На странице капча
На странице капча
На странице капча
На странице капча
На странице капча
На странице капча
На странице капча
На странице капча
На странице ошибка
На странице ошибка
На странице капча
На странице капча
На странице капча
На странице капча
На странице капча
На странице капча
На странице ошибка
Н

KeyboardInterrupt: 

In [52]:
with open('cian_data' + str(12), 'wb') as f:
    pickle.dump(data, f)

In [53]:
len(cur_hrefs)

18

In [54]:
for hr in bad_hrefs:
    cur_hrefs.remove(hr)

In [55]:
len(cur_hrefs)

18

In [ ]:
len(data)

In [57]:
data[-1]

{'href': 'https://astrahan.cian.ru/rent/commercial/163707888/',
 'Адрес': 'Астраханская область,\n    \n        \n        Астрахань,\n    \n        \n        ул. Дзержинского',
 'Координаты': '[47.997821,46.348634]',
 'Площадь:': '30\xa0м²',
 'Цена1': '9\xa0900\xa0руб. в\xa0месяц',
 'Цена2': '3\xa0960\xa0руб. за\xa0м² в\xa0год'}

In [32]:
data[-200]

{'href': 'https://samara.cian.ru/rent/commercial/160903674/',
 'Адрес': 'Самарская область,\n    \n        \n        Самара,\n    \n        \n        район Железнодорожный,\n    \n        \n        Аэродромная ул.,\n    \n        \n        45А',
 'Арендные каникулы:': 'да',
 'Вентиляция:': 'естественная',
 'Высота потолков:': '3\xa0м',
 'Год постройки:': '2007',
 'Доступ:': 'свободный',
 'Категория:': 'действующий',
 'Класс:': 'C',
 'Кондиционирование:': 'местное',
 'Координаты': '[50.188309,53.190267]',
 'Лифты:': '2',
 'Мебель:': 'нет',
 'Минимальный срок аренды:': '11\xa0мес',
 'Обеспечительный платеж:': '14\xa0065\xa0руб.',
 'Общая площадь:': '12\xa0000\xa0м²',
 'Отопление:': 'центральное',
 'Парковка:': 'наземная',
 'Планировка:': 'кабинетная',
 'Площадь:': '29\xa0м²',
 'Помещение:': 'свободно',
 'Предоплата:': '1\xa0мес',
 'Система пожаротушения:': 'гидрантная',
 'Состояние:': 'офисная отделка',
 'Срок аренды:': 'длительный',
 'Тип аренды:': 'прямая',
 'Участок:': '0,13\xa0га (в 

## Cоздание таблички

In [68]:
%%time
# В эту табличку будем собирать данные
df = pd.DataFrame( )
k = 0 # Это номера наблюдений

for item in tqdm_notebook(data):
    k = k + 1
    # грузим новое наблюдение
    df1 = pd.DataFrame.from_dict(item,orient='index')
    # присваиваем этому наблюдению номер
    df1.columns =[k]
    # закидываем его в итоговую таблицу
    df = df.join(df1, how='outer')   

df = df.T  # Для удобства транспонируем таблицу        
df.head() 


CPU times: user 6h 40min 12s, sys: 5min 42s, total: 6h 45min 55s
Wall time: 6h 41min 52s


In [ ]:
# ПОЧЕМУ ТАК ДОЛГО?! 
df.shape

In [71]:
pd.DataFrame.to_csv(df,'CIAN_data_dirty.csv')

In [4]:
df = pd.read_csv('CIAN_data_dirty.csv')

/usr/local/lib/python3.5/dist-packages/IPython/core/interactiveshell.py:2717: DtypeWarning: Columns (6,7,8,13,15,22,24,25,28,40,42,45,47,48,49,55) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


## Очистка таблички 

In [5]:
df.columns

Index(['Unnamed: 0', 'href', 'Адрес', 'Арендаторы:', 'Арендные каникулы:',
       'Вентиляция:', 'Витринные окна:', 'Ворота:', 'Вход:',
       'Высота потолков:', 'Год постройки:', 'Дата освобождения:',
       'Девелопер:', 'Дни работы:', 'Доступ:', 'Залог:', 'Категория:',
       'Класс:', 'Кол-во мокрых точек:', 'Количество мест:',
       'Кондиционирование:', 'Координаты', 'Линия домов:', 'Лифты:',
       'Масштаб комплекса:', 'Материал пола:', 'Мебель:',
       'Минимальный срок аренды:', 'Название ГСК:', 'Номер налоговой:',
       'Обеспечительный платеж:', 'Общая площадь:', 'Отопление:', 'Парковка:',
       'Планировка:', 'Площадь:', 'Помещение:', 'Предоплата:',
       'Система пожаротушения:', 'Состояние:', 'Специализация помещения:',
       'Срок аренды:', 'Статус:', 'Стоимость:', 'Тип аренды:', 'Тип гаража:',
       'Тип здания:', 'Тип парковки:', 'Тип помещения:', 'Тип:',
       'Траволаторы:', 'Управляющая компания:', 'Участок:', 'Цена1', 'Цена2',
       'Часы работы:', 'Элек

In [6]:
df_clean = pd.DataFrame( )
df_clean['href'] = df['href']
df_clean['Город'] = [item.split('\n    \n        \n        ')[1] for item in df['Адрес']]
df_clean['Субъект'] = [item.split('\n    \n        \n        ')[0] for item in df['Адрес']]
# Для Москвы и питера в город попало метро 

print('Класс не указан у', sum(df['Класс:'].isnull()), 'из', df.shape[0])
cl = [ ]
for item in df['Класс:']:
    try:
        cl.append(item[0])
    except:
        cl.append(item)
df_clean['Класс'] = cl

plos = [ ]
for item in df['Площадь:']:
    try:
        plos.append(item.split('\xa0')[-2])
    except:
        plos.append(item)

        
df_clean['Площадь'] = plos
df_clean['Цена в месяц'] = [item.split('\xa0')[0] + item.split('\xa0')[1] for item in df['Цена1']]
df_clean['Цена в год за м2'] = [item.split('\xa0')[0] + item.split('\xa0')[1] for item in df['Цена2']]
df_clean.head()

Класс не указан у 11215 из 16746


,href,Город,Субъект,Класс,Площадь,Цена в месяц,Цена в год за м2
0,https://krasnodar.cian.ru/rent/commercial/1576...,"Краснодар,","Краснодарский край,",NaN,890,17000,12000
1,https://krasnodar.cian.ru/rent/commercial/1653...,"Краснодар,","Краснодарский край,",NaN,21,14700,8400
2,https://krasnodar.cian.ru/rent/commercial/1663...,"Краснодар,","Краснодарский край,",NaN,60,20100,4020
3,https://krasnodar.cian.ru/rent/commercial/7763...,"Краснодар,","Краснодарский край,",B,"36,8",22080,7200
4,https://anapa.cian.ru/rent/commercial/165898822/,"Анапский район,","Краснодарский край,",NaN,37,28490,9240


In [12]:
# Вектор int_city задаётся ниже 

def pokaz(ct_df):
    s = 0
    n = 0
    for item in ct_df['Цена в год за м2']:
        try:
            s += int(item)
            n += 1
        except:
            # print(item)
            pass
    mx = 0
    for item in ct_df['Площадь']:
        try:
            cur = float(item.replace(',','.'))
            if cur > mx:
                mx = cur
        except:
            #print(item)
            pass
        
    sstr_pr = [ ]
    for item in ct_df['Цена в год за м2']:
        try:
            sstr_pr.append(float(item))
        except:
            #print(item)
            pass
    med = np.median(sstr_pr)
    med = med/12
    #print('Максимальная площадь офиса',mx)    
    #print('Средняя цена за год',s/(n+0.0001))
    ss = s/(n+0.0001)
    ss = ss/12
    return mx,med,ss,n

def counter(city,tp='Город'):
    ct_df = df_clean[df_clean[tp] == city]
    mx,med,s,n = pokaz(ct_df)
    
    mxa,meda,sa,na = pokaz(ct_df[ct_df['Класс'] == 'A'])
    mxb,medb,sb,nb = pokaz(ct_df[ct_df['Класс'] == 'B'])
    mxc,medc,sc,nc = pokaz(ct_df[ct_df['Класс'] == 'C'])
    mini_df = pd.DataFrame([city,n,mx,s,med,na,mxa,sa,meda,nb,mxb,sb,medb,nc,mxc,sc,medc]).T
    mini_df.columns = ['Город','Всего','Максима площ', 'Ср цена за год','Медианная цена за год',\
                                'Всего А','Максим площ А','Ср цена за год А','Медианная цена за год A',\
                                'Всего В','Максим площ В','Ср цена за год В','Медианная цена за год B',\
                                'Всего С','Максим площ C','Ср цена за год С','Медианная цена за год C']
    return(mini_df)

itog_cian_df = pd.DataFrame( )
for cty in int_city[:2]:
    itog_cian_df = itog_cian_df.append(counter(cty+',',tp='Субъект'))
for cty in int_city[2:]:
    itog_cian_df = itog_cian_df.append(counter(cty+','))

/usr/local/lib/python3.5/dist-packages/numpy/core/fromnumeric.py:2909: RuntimeWarning: Mean of empty slice.
  out=out, **kwargs)
/usr/local/lib/python3.5/dist-packages/numpy/core/_methods.py:80: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)


In [13]:
# pd.DataFrame.to_csv(itog_cian_df,'CIAN_clean_itog.csv')
writer = pd.ExcelWriter('output.xlsx')
itog_cian_df.to_excel(writer,'Sheet1')
writer.save()

In [8]:
int_city = ['Москва',
'Санкт-Петербург',
'Новосибирск',
'Екатеринбург',
'Нижний Новгород',
'Казань',
'Челябинск',
'Омск',
'Самара',
'Ростов-на-Дону',
'Уфа',
'Красноярск',
'Пермь',
'Воронеж',
'Волгоград',
'Краснодар',
'Саратов',
'Тюмень',
'Тольятти',
'Ижевск',
'Барнаул',
'Ульяновск',
'Иркутск',
'Хабаровск',
'Ярославль',
'Владивосток',
'Махачкала',
'Томск',
'Оренбург',
'Кемерово',
'Новокузнецк',
'Рязань',
'Астрахань',
'Набережные Челны',
'Пенза',
'Липецк',
'Киров',
'Чебоксары',
'Тула',
'Калининград',
'Балашиха',
'Курск',
'Ставрополь',
'Улан-Удэ',
'Тверь',
'Магнитогорск',
'Сочи',
'Иваново',
'Брянск']

Названия колонок для таблички: 

Город (численность на 01.01.2017 > 400 тыс. Чел.)	Найдено объявлений в городе	Максимальная площадь офиса, м2	Средняя стоимость аренды офиса, м2/ месяц	Медианная цена аренды офиса, м2/месяц	Найдено объявлений класса А в городе	Максимальная площадь офиса класса А, м2	Средняя стоимость аренды офиса класса А, м2/ месяц	Медианная цена аренды офиса класса А, м2/месяц	Найдено объявлений класа В в городе	Максимальная площадь офиса класса В, м2	Средняя стоимость аренды офиса класса В, м2/ месяц	Медианная цена аренды офиса класса В, м2/месяц	Найдено объявлений класса С в городе	Максимальная площадь офиса класса С, м2	Средняя стоимость аренды офиса класса С, м2/ месяц	Медианная цена аренды офиса класса С, м2/месяц
